In [1]:
"""Before starting, check that vanilla and JIT-compiled functions give the same results...
The proper test code is already part of the library file, so just run it.
"""
%run secondlayer.py

Performing some poor man's (not so unit) testing.

Testing 'gdummyv' implementations: 	OK!
Testing 'gdummynv' implementations: 	OK!
Testing 'gdummynvana' implementations: 	OK!
Testing 'compute_error_v' implementations: 	OK!
Testing 'compute_currents' implementations: 	OK!

If you are reading this, then everything seems to have run nicely :).


## Benchmarking `gdummy*`-functions 

In [2]:
import numpy as np
from secondlayer import (_gdummyv_vanilla, _gdummyv_numpy, _gdummyv_jit,
                         _gdummynv_vanilla, _gdummynv_numpy, _gdummynv_jit,
                         _gdummynvana_vanilla, _gdummynvana_numpy, _gdummynvana_jit)

# ##################
# Helper function(s)
def get_dummy_data(seed=None):
    _prng = np.random.RandomState(seed)  # for reproducibility
    m = 4*10  # number of features
    n = 3*10  # number classes
    gmin, gmax = 1e-8, 1e-6
    g = 256  # number of writable levels
    G = _prng.uniform(low=gmin, high=gmax, size=(2*m + 2, n))
    error = _prng.random_sample(size=G.shape)
    deltas = np.zeros(n)
    gmins = _prng.normal(loc=gmin, scale=0.2 * gmin, size=G.shape)
    gmaxs = _prng.normal(loc=gmax, scale=0.2 * gmax, size=G.shape)
    memarray = np.dstack((gmaxs, gmins))

    return G, gmax, gmin, g, m, n, error, deltas, memarray


# Run the JIT version(s) once to compile it (them)

G0, _, _, g, m, n, error, _, memarray = get_dummy_data()
G0 = _gdummyv_jit(G0, g, m, n, error, memarray)

G1, gmax, gmin, g, m, n, error, _, _ = get_dummy_data()
G1 = _gdummynv_jit(G1, gmax, gmin, g, m, n, error)
              
G2, gmax, gmin, g, m, n, error, deltas, _ = get_dummy_data()
G2 = _gdummynvana_jit(G2, gmax, gmin, g, m, n, error, deltas)

In [3]:
print("### Benchmarking `gdummyv` implementations ###")
seed = np.random.randint(2134)

for func, case in ((_gdummyv_vanilla, "Vanilla:"),
                   (_gdummyv_numpy, "Numpy-only:"),
                   (_gdummyv_jit, "JIT-compiled:")):
    G0, _, _, g, m, n, error, _, memarray = get_dummy_data(seed)
    print(case, end="\t")
    %timeit func(G0, g, m, n, error, memarray)

### Benchmarking `gdummyv` implementations ###
Vanilla:	10 loops, best of 3: 22.1 ms per loop
Numpy-only:	10000 loops, best of 3: 103 µs per loop
JIT-compiled:	10000 loops, best of 3: 22 µs per loop


In [4]:
print("### Benchmarking `gdummynv` implementations ###")
seed = np.random.randint(3241)

for func, case in ((_gdummynv_vanilla, "Vanilla:"),
                   (_gdummynv_numpy, "Numpy-only:"),
                   (_gdummynv_jit, "JIT-compiled:")):
    G1, gmax, gmin, g, m, n, error, _, _ = get_dummy_data(seed)
    print(case, end="\t")
    %timeit func(G1, gmax, gmin, g, m, n, error)

### Benchmarking `gdummynv` implementations ###
Vanilla:	100 loops, best of 3: 9.19 ms per loop
Numpy-only:	The slowest run took 5.34 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 54.7 µs per loop
JIT-compiled:	100000 loops, best of 3: 10.5 µs per loop


In [5]:
print("### Benchmarking `gdummynvana` implementations ###")
seed = np.random.randint(4231)

for func, case in ((_gdummynvana_vanilla, "Vanilla:"),
                   (_gdummynvana_numpy, "Numpy-only:"),
                   (_gdummynvana_jit, "JIT-compiled:")):
    G2, gmax, gmin, g, m, n, error, deltas, _ = get_dummy_data(seed)
    print(case, end="\t")
    %timeit func(G2, gmax, gmin, g, m, n, error, deltas)

# Remark: why is the JIT-compiled version so slow in this case?!

### Benchmarking `gdummynvana` implementations ###
Vanilla:	100 loops, best of 3: 10.8 ms per loop
Numpy-only:	10000 loops, best of 3: 92 µs per loop
JIT-compiled:	100 loops, best of 3: 4.16 ms per loop


## Benchmarking `compute_error_*`-functions 

In [6]:
import numpy as np
from secondlayer import (_compute_error_v_vanilla,
                         _compute_error_v_numpy,
                         _compute_error_v_jit)

# ##################
# Helper function(s)
    
def get_error_data(seed=None):
    """Return dummy arguments for 'compute_error_v*'-type functions. """
    _prng = np.random.RandomState(seed)  # for reproducibility
    nfts, ncls = 4*100, 3*100  # NB: bigger than in the iris case
    xit = _prng.uniform(low=-1, high=1, size=nfts)
    yit = np.zeros(ncls) - 1
    yit[-1] = 1
    outs = np.copy(yit)
    outs += np.random.uniform(low=-0.2, high=0.2, size=len(outs))
    outs[0] *= np.sign(outs[0]*yit[0])  # ensure correct answer
    outs[1] *= -np.sign(outs[1]*yit[1])  # ensure wrong answer

    return xit, yit, outs


# Run the JIT version(s) once to compile it (them)

xit, yit, outs = get_error_data()
error = _compute_error_v_jit(xit, yit, outs)

In [7]:
print("### Benchmarking `compute_error_v` implementations ###")
seed = np.random.randint(3124)

for func, case in ((_compute_error_v_vanilla, "Vanilla:"),
                   (_compute_error_v_numpy, "Numpy-only:"),
                   (_compute_error_v_jit, "JIT-compiled:")):
    my_args = get_error_data(seed)
    print(case, end="\t")
    %timeit func(*my_args)

### Benchmarking `compute_error_v` implementations ###
Vanilla:	100 loops, best of 3: 2.89 ms per loop
Numpy-only:	1000 loops, best of 3: 473 µs per loop
JIT-compiled:	1000 loops, best of 3: 215 µs per loop


## Benchmarking `compute_currents_*`-functions 

In [8]:
import numpy as np
from secondlayer import (_compute_currents_vanilla,
                         _compute_currents_numpy,
                         _compute_currents_jit)

# ##################
# Helper function(s)

def get_current_data(seed=None):
    """Return dummy arguments for 'compute_currents*'-type functions. """
    _prng = np.random.RandomState(seed)  # for reproducibility
    nfts, ncls = 4*10, 3*10  # NB: bigger than in the iris case
    xit = _prng.uniform(low=-1, high=1, size=nfts)
    gmin, gmax = 1e-8, 1e-6
    G = _prng.uniform(low=gmin, high=gmax, size=(2*nfts + 2, ncls))

    return xit, G

# Run the JIT version(s) once to compile it (them)

xit, G = get_current_data()
currents = _compute_currents_jit(xit, G)

In [9]:
print("### Benchmarking `compute_error_v` implementations ###")
seed = np.random.randint(2233)

for func, case in ((_compute_currents_vanilla, "Vanilla:"),
                   (_compute_currents_numpy, "Numpy-only:"),
                   (_compute_currents_jit, "JIT-compiled:")):
    my_args = get_current_data(seed)
    print(case, end="\t")
    %timeit func(*my_args)

### Benchmarking `compute_error_v` implementations ###
Vanilla:	The slowest run took 5.63 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 43.1 µs per loop
Numpy-only:	The slowest run took 13.38 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 17 µs per loop
JIT-compiled:	The slowest run took 4.91 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.57 µs per loop
